In [1]:
# import packages
import pandas as pd
import pickle as pkl
import os

In [2]:
from sqlalchemy import create_engine

def postgresql_engine(user, pwd, host, port, dbname):
    # Need pyycopg2-binary package
    sql_engine = create_engine('postgres://' + user + ':' + pwd + '@' + host + ':' + port + '/' + dbname, echo=False)
    return sql_engine

In [3]:
# DB username and password
import getpass

user = getpass.getpass()
pwd = getpass.getpass()

In [4]:
# misc db parameters
host= 'adds-postgres-dev.cfgztrijqgvp.us-east-1.rds.amazonaws.com'
dbname= 'musiclab'
port= '5432'

In [10]:
# get callout research for songs released in the past 2 years
data_query = '''
Select sv.song_name || '( ' || sv.artist_name || ' )' as song_artist, s.call_letters, brks.breakout_name, rdsa.demo_category, rdsa.week_dt,
rdsa.pop_all, CAST(rdsa.pop_predicted AS DECIMAL(5,2)), CASE WHEN (rdsa.pop_all > wobble_upper_threshold
    OR rdsa.pop_all < wobble_lower_threshold) then 1 else 0 end as wobble_flag
from dbo.rr_demo_scores_adds as rdsa
left join data.songs_v as sv
on sv.mediabase_id=rdsa.mediabase_id
left join data.stations as s
on s.station_id=rdsa.station_id
left join
(
Select distinct cmm_station_calls, breakout_id, REPLACE(breakout_name, '*', '') as breakout_name
from data.cmm
where format='CHR'
UNION
Select distinct cmm_station_calls, -1, 'Non-Core'
from data.cmm
where format='CHR'
UNION
Select distinct cmm_station_calls, -2, 'Female-Other'
from data.cmm
where format='CHR'
) as brks
on brks.breakout_id = rdsa.breakout_id
and brks.cmm_station_calls=s.call_letters
where rdsa.pop_all is not null
'''

In [11]:
engine = postgresql_engine(user, pwd, host, port, dbname)
with engine.connect() as con:
    with con.connect():
        df = pd.read_sql(data_query, con=con)

In [12]:
df[~pd.isna(df['pop_predicted'])].groupby(['call_letters', 'demo_category', 'breakout_name']).agg({'song_artist': len})

song_artist
call_letters demo_category breakout_name             
KBKS-FM      Age           Old                     25
                           Young                   25
             Core-Cume     Core                    25
                           Non-Core                25
             Gender        F (18-24)               25
...                                               ...
WXKS-FM      Core-Cume     Non-Core                24
             Gender        F (18-24)               24
                           Female-Other            24
             Race          Hispanic                48
                           White                   48

[108 rows x 1 columns]

In [13]:
df[~pd.isna(df['pop_predicted'])].groupby(['call_letters', 'song_artist', 'week_dt']).agg(len)

breakout_name  \
call_letters song_artist                                    week_dt                     
KBKS-FM      About Damn Time( LIZZO )                       2022-11-27              8   
             As It Was( HARRY STYLES )                      2022-11-27              8   
             Bad Habit( STEVE LACY )                        2022-11-27              8   
             Best Friend f/Doja Cat( SAWEETIE )             2022-11-27              8   
             Big Energy( LATTO )                            2022-11-27              8   
...                                                                               ...   
WXKS-FM      Sweetest Pie( MEGAN THEE STALLION & DUA LIPA ) 2022-11-27             10   
             Unholy( SAM SMITH & KIM PETRAS )               2022-11-27             10   
             Vegas( DOJA CAT )                              2022-11-27             10   
             Woman( DOJA CAT )                              2022-11-27             10   
             good 4 u( OLIVIA RODRIGO )                     2022-11-27             10   

                                                                        demo_category  \
call_letters song_artist                                    week_dt                     
KBKS-FM      About Damn Time( LIZZO )                       2022-11-27              8   
             As It Was( HARRY STYLES )                      2022-11-27              8   
             Bad Habit( STEVE LACY )                        2022-11-27              8   
             Best Friend f/Doja Cat( SAWEETIE )             2022-11-27              8   
             Big Energy( LATTO )                            2022-11-27              8   
...                                                                               ...   
WXKS-FM      Sweetest Pie( MEGAN THEE STALLION & DUA LIPA ) 2022-11-27             10   
             Unholy( SAM SMITH & KIM PETRAS )               2022-11-27             10   
             Vegas( DOJA CAT )                              2022-11-27             10   
             Woman( DOJA CAT )                              2022-11-27             10   
             good 4 u( OLIVIA RODRIGO )                     2022-11-27             10   

                                                                        pop_all  \
call_letters song_artist                                    week_dt               
KBKS-FM      About Damn Time( LIZZO )                       2022-11-27        8   
             As It Was( HARRY STYLES )                      2022-11-27        8   
             Bad Habit( STEVE LACY )                        2022-11-27        8   
             Best Friend f/Doja Cat( SAWEETIE )             2022-11-27        8   
             Big Energy( LATTO )                            2022-11-27        8   
...                                                                         ...   
WXKS-FM      Sweetest Pie( MEGAN THEE STALLION & DUA LIPA ) 2022-11-27       10   
             Unholy( SAM SMITH & KIM PETRAS )               2022-11-27       10   
             Vegas( DOJA CAT )                              2022-11-27       10   
             Woman( DOJA CAT )                              2022-11-27       10   
             good 4 u( OLIVIA RODRIGO )                     2022-11-27       10   

                                                                        pop_predicted  \
call_letters song_artist                                    week_dt                     
KBKS-FM      About Damn Time( LIZZO )                       2022-11-27              8   
             As It Was( HARRY STYLES )                      2022-11-27              8   
             Bad Habit( STEVE LACY )                        2022-11-27              8   
             Best Friend f/Doja Cat( SAWEETIE )             2022-11-27              8   
             Big Energy( LATTO )                            2022-11-27              8   
...                                                  

In [35]:
df_out = df[~pd.isna(df['pop_predicted'])].pivot_table(index=['call_letters', 'song_artist'], columns=['breakout_name'], values=['pop_predicted', 'pop_all', 'wobble_flag'])

In [36]:
df_out.columns = [i[0] + '_' + i[1].replace(' ', '_') for i in df_out.columns]

In [37]:
df_out.columns

Index(['pop_all_AA', 'pop_all_Core', 'pop_all_F_(18-24)',
       'pop_all_Female-Other', 'pop_all_Hispanic', 'pop_all_Non-Core',
       'pop_all_Old', 'pop_all_WAO', 'pop_all_White', 'pop_all_Young',
       'pop_predicted_AA', 'pop_predicted_Core', 'pop_predicted_F_(18-24)',
       'pop_predicted_Female-Other', 'pop_predicted_Hispanic',
       'pop_predicted_Non-Core', 'pop_predicted_Old', 'pop_predicted_WAO',
       'pop_predicted_White', 'pop_predicted_Young', 'wobble_flag_AA',
       'wobble_flag_Core', 'wobble_flag_F_(18-24)', 'wobble_flag_Female-Other',
       'wobble_flag_Hispanic', 'wobble_flag_Non-Core', 'wobble_flag_Old',
       'wobble_flag_WAO', 'wobble_flag_White', 'wobble_flag_Young'],
      dtype='object')

In [38]:
df_out['gender_perc_diff'] = (1 - df_out['pop_predicted_Female-Other']/df_out['pop_all_F_(18-24)'])

In [39]:
df_out['core_perc_diff'] = (1 - df_out['pop_predicted_Non-Core']/df_out['pop_predicted_Core'])

In [40]:
df_out['Hispanic_perc_diff'] = (1- df_out['pop_predicted_Hispanic']/(df_out['pop_predicted_White'].combine_first(df_out['pop_all_WAO'])))

In [41]:
df_out['AA_perc_diff'] = (1- df_out['pop_predicted_AA']/(df_out['pop_predicted_White'].combine_first(df_out['pop_all_WAO'])))

In [51]:
df_out['age_perc_diff'] = (1- df_out['pop_predicted_Young']/(df_out['pop_predicted_Old']))

In [52]:
out_stations = pd.unique(df_out.index.get_level_values(0))

In [54]:
with pd.ExcelWriter('Score_Gaps_by_Demographic_122022.xlsx') as writer:
    for stat in out_stations:
        idx = df_out[df_out.index.get_level_values(0) == stat].index
        df_out.loc[idx].to_excel(writer, sheet_name=stat)